In [ ]:
import torch
from torchvision import datasets, transforms
from tqdm import tqdm

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

print(trainloader.dataset.data.shape) # 60000 * 28  28

dataiter = iter(trainloader)
images, labels = next(dataiter)
print(images.shape[-1])
images = torch.cat((images, images, images), 1)
print(images.shape)
# print("images shape:", images.shape, "labels shape:", labels.shape)


In [ ]:
import torch
import torchvision.models as models

model = models.resnet18()
print(model)

# Input shape
input_shape = (3, 224, 224)
x = torch.randn(1, *input_shape)
print(f"Input shape: {x.shape}")

# Summary
from torchsummary import summary
summary(model, input_shape)


In [3]:
from domain import Network
from dataset import mnist_train_loader, mnistm_train_loader
import torch
import torch.nn as nn

train_source, train_target = mnist_train_loader, mnistm_train_loader

net = Network(train_source, train_target)

print(torch.cuda.device_count())
if torch.cuda.device_count() > 1:
    layout = list(net.model.module.children())
else:
    layout = list(net.model.children())

model = layout[:-1]
model = nn.Sequential(*model)
print(model)
# print(nn.Sequential(*list(net.model.children())))


2
Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): C

In [ ]:
import torch.nn as nn
import torchvision.models as models

# 사전 훈련된 ResNet 모델 로드
resnet = models.resnet18(pretrained=True)

# 모델의 자식 레이어들을 리스트로 변환

# 마지막 완전 연결 레이어를 제외한 나머지 레이어들 선택
feature_extractor_layers = list(resnet.children())[:-1]  # 마지막 레이어 제외

# 선택된 레이어들로 새로운 Sequential 모델 생성
feature_extractor = nn.Sequential(*feature_extractor_layers)
print(feature_extractor)

# 결과 확인
# for layer in feature_extractor:
#     print(layer)
